###Config. Iniciais:
*   Importo as bibliotecas;
*   Monto o drive e crio os data_frames

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from scipy.stats import spearmanr #pegando apenas a funcao spearmanr
import re # Para processar a string de duração ISO 8601

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

folder_path = '/content/drive/MyDrive/red_pill_analises/'
caminho_canais = 'data_base/dados_limpos/canais_limpos.csv'
caminho_videos = 'data_base/dados_limpos/videos_limpos.csv'
caminho_comentarios = 'data_base/dados_limpos/comentarios_limpos.csv'

df_canais_limpos = pd.read_csv(folder_path + caminho_canais)
df_videos_limpos = pd.read_csv(folder_path + caminho_videos)
df_comentarios_limpos = pd.read_csv(folder_path + caminho_comentarios)

Mounted at /content/drive


In [ ]:
df_comentarios_limpos.columns

Index(['Unnamed: 0.6', 'Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0.3',
       'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'text', 'published_at',
       'updated_at', 'like_count', 'viewer_rating', 'can_rate', 'is_reply',
       'parent_id', 'id_video_anonimizado', 'id_canal_anonimizado',
       'id_autor_anonimizado', 'id_comentario_anonimizado', 'emojis',
       'hashtags'],
      dtype='object')

###Coef. de Spearman:


*   Defino a função para cal. o coef;
*   Função para padronizar o tempo;
*   Limpa e organiza as colunas para serem usadas na análise;
*   Puxa a tabelona;
*   Análise de correlação entre a frequência de postagem, a duração dos vídeos e o sucesso (medido por engajamento e inscritos) de cada canal;





In [ ]:
# Coef de Spearman
def spearman(coluna_1, coluna_2, nome1, nome2):
  #calc. o coef. de spearman
  coeficiente_spearman, p_valor_spearman = spearmanr(
    coluna_1,
    coluna_2)

  print(f"Coeficiente de Correlação de Spearman entre '{nome1}' e '{nome2}': {coeficiente_spearman:.3f}")

# Transformar em segundos
def iso_to_seconds(duration_str):
  if not isinstance(duration_str, str): # Verifica se o valor é uma string
        return None
  duration_match = re.match(r'PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?', duration_str)
  if not duration_match:
        return None # Retorna None para formatos que não correspondem
  hours = int(duration_match.group(1)) if duration_match.group(1) else 0
  minutes = int(duration_match.group(2)) if duration_match.group(2) else 0
  seconds = int(duration_match.group(3)) if duration_match.group(3) else 0
  return hours * 3600 + minutes * 60 + seconds #conversão para segundos

In [ ]:
# VÍDEOS
coluna_likes = 'like_count'
coluna_views = 'view_count'
coluna_comentarios = 'comment_count'
coluna_duracao = 'duration'
df_videos_limpos['duration_seconds'] = df_videos_limpos[coluna_duracao].apply(iso_to_seconds)
df_analise = df_videos_limpos[[coluna_likes, coluna_views, coluna_comentarios, 'duration_seconds']].copy()
df_analise[coluna_comentarios] = pd.to_numeric(df_analise[coluna_comentarios], errors='coerce')
df_analise[coluna_likes] = pd.to_numeric(df_analise[coluna_likes], errors='coerce')
df_analise[coluna_views] = pd.to_numeric(df_analise[coluna_views], errors='coerce')
df_analise.dropna(subset=[coluna_likes, coluna_views, coluna_comentarios, 'duration_seconds'], inplace=True)

# Comentários vs. Duração dos vídeos
spearman(df_analise[coluna_comentarios], df_analise['duration_seconds'], 'quantidade de comentários', 'duração')
# Visualizações vs. Duração dos vídeos
spearman(df_analise[coluna_views], df_analise['duration_seconds'], 'quantidade de visualizações', 'duração')
# Likes vs. Duração dos vídeos
spearman(df_analise[coluna_likes], df_analise['duration_seconds'], 'quantidade de likes', 'duração')

Coeficiente de Correlação de Spearman entre 'quantidade de comentários' e 'duração': 0.210
Coeficiente de Correlação de Spearman entre 'quantidade de visualizações' e 'duração': -0.059
Coeficiente de Correlação de Spearman entre 'quantidade de likes' e 'duração': 0.127


In [ ]:
# CANAL
tabelona = pd.read_csv(folder_path + 'data_base/tabelona_final.csv')
tabelona.columns

Index(['id_canal_anonimizado', 'Inscritos', 'Vídeos coletados',
       'Meses de atividade', 'Média vídeos/mês (dp)', 'Duração média (s) (dp)',
       'Total visualizações', 'Média visualizações (dp)', 'Total curtidas',
       'Média likes (dp)', 'Total comentários', 'Média comentários (dp)'],
      dtype='object')

In [ ]:
coluna_videos_por_mes = tabelona['Média vídeos/mês (dp)'].str.split(' ').str[0].astype(float)
coluna_total_comentarios = tabelona['Total comentários']
coluna_total_visualizacoes = tabelona['Total visualizações']
coluna_total_likes = tabelona['Total curtidas']
coluna_duracao_media = tabelona['Duração média (s) (dp)'].str.split(' ').str[0].astype(float)
coluna_media_comentarios = tabelona['Média comentários (dp)'].str.split(' ').str[0].astype(float)
coluna_media_visualizacoes = tabelona['Média visualizações (dp)'].str.split(' ').str[0].astype(float)
coluna_media_likes = tabelona['Média likes (dp)'].str.split(' ').str[0].astype(float)
coluna_inscritos = tabelona['Inscritos']

spearman(coluna_videos_por_mes, coluna_total_comentarios, 'quantidade de vídeos por mês', 'total de comentários')
spearman(coluna_videos_por_mes, coluna_total_visualizacoes, 'quantidade de vídeos por mês', 'total de visualizações')
spearman(coluna_videos_por_mes, coluna_total_likes, 'quantidade de vídeos por mês', 'total de likes')
spearman(coluna_videos_por_mes, coluna_duracao_media, 'quantidade de vídeos por mês', 'duração média')
spearman(coluna_videos_por_mes, coluna_media_comentarios, 'quantidade de vídeos por mês', 'média de comentários')
spearman(coluna_videos_por_mes, coluna_media_visualizacoes, 'quantidade de vídeos por mês', 'média de visualizações')
spearman(coluna_videos_por_mes, coluna_media_likes, 'quantidade de vídeos por mês', 'média de likes')
spearman(coluna_videos_por_mes, coluna_inscritos, 'quantidade de vídeos por mês', 'inscritos')


spearman(coluna_duracao_media, coluna_total_comentarios, 'duração média', 'total de comentários')
spearman(coluna_duracao_media, coluna_total_visualizacoes, 'duração média', 'total de visualizações')
spearman(coluna_duracao_media, coluna_total_likes, 'duração média', 'total de likes')
spearman(coluna_duracao_media, coluna_media_comentarios, 'duração média', 'média de comentários')
spearman(coluna_duracao_media, coluna_media_visualizacoes, 'duração média', 'média de visualizações')
spearman(coluna_duracao_media, coluna_media_likes, 'duração média', 'média de likes')
spearman(coluna_duracao_media, coluna_inscritos, 'duração média', 'inscritos')

Coeficiente de Correlação de Spearman entre 'quantidade de vídeos por mês' e 'total de comentários': 0.238
Coeficiente de Correlação de Spearman entre 'quantidade de vídeos por mês' e 'total de visualizações': 0.445
Coeficiente de Correlação de Spearman entre 'quantidade de vídeos por mês' e 'total de likes': 0.368
Coeficiente de Correlação de Spearman entre 'quantidade de vídeos por mês' e 'duração média': -0.424
Coeficiente de Correlação de Spearman entre 'quantidade de vídeos por mês' e 'média de comentários': -0.207
Coeficiente de Correlação de Spearman entre 'quantidade de vídeos por mês' e 'média de visualizações': 0.214
Coeficiente de Correlação de Spearman entre 'quantidade de vídeos por mês' e 'média de likes': 0.084
Coeficiente de Correlação de Spearman entre 'quantidade de vídeos por mês' e 'inscritos': 0.062
Coeficiente de Correlação de Spearman entre 'duração média' e 'total de comentários': -0.043
Coeficiente de Correlação de Spearman entre 'duração média' e 'total de vis

In [ ]:
tabelona.sort_values(by = 'Vídeos coletados', ascending=False).reset_index(drop=True)

,id_canal_anonimizado,Inscritos,Vídeos coletados,Meses de atividade,Média vídeos/mês (dp),Duração média (s) (dp),Total visualizações,Média visualizações (dp),Total curtidas,Média likes (dp),Total comentários,Média comentários (dp)
0,canal_1,325000,2134,18,118.6 (48.3),94.7 (107.1),102759571,48153.5 (165719.5),8833125,4139.2 (14375.0),332305,155.7 (332.5)
1,canal_2,211000,1698,15,113.2 (42.1),75.6 (42.5),141435162,83295.1 (301671.7),11832309,6968.4 (23493.0),387503,228.2 (526.9)
2,canal_3,77200,1498,18,83.2 (22.9),116.1 (213.1),26280517,17543.7 (104589.6),1098259,733.2 (3632.5),25418,17.0 (87.2)
3,canal_4,17000,1277,13,98.2 (62.6),166.7 (113.1),5802479,4543.8 (95742.2),232493,182.1 (3220.9),16600,13.0 (37.4)
4,canal_5,423000,1147,18,63.7 (13.1),52.3 (28.4),78657146,68576.4 (435370.4),5002954,4361.8 (20002.6),143482,125.1 (419.5)
5,canal_6,51100,1065,18,59.2 (18.3),96.3 (280.1),11530336,10826.6 (33455.8),793787,745.3 (1352.3),26679,25.1 (54.1)
6,canal_7,101000,992,17,58.4 (27.5),108.3 (252.6),126248207,127266.3 (732266.8),101651,102.5 (2254.8),59881,60.4 (254.6)
7,canal_8,46700,960,9,106.7 (56.2),72.8 (38.3),27007213,28132.5 (188179.9),1821735,1897.6 (13885.8),45674,47.6 (199.5)
8,canal_9,1960,808,11,73.5 (45.1),128.7 (103.4),448015,554.5 (2491.7),22555,27.9 (111.6),3826,4.7 (32.7)
9,canal_10,320000,709,18,39.4 (23.0),5067.2 (4430.6),33125726,46721.8 (217032.8),2137261,3014.5 (15724.0),132537,186.9 (457.7)


In [ ]:
tabelona.columns

Index(['id_canal_anonimizado', 'Inscritos', 'Vídeos coletados',
       'Meses de atividade', 'Média vídeos/mês (dp)', 'Duração média (s) (dp)',
       'Total visualizações', 'Média visualizações (dp)', 'Total curtidas',
       'Média likes (dp)', 'Total comentários', 'Média comentários (dp)'],
      dtype='object')

In [ ]:
total_comentarios = len(df_comentarios_limpos)
total_views = df_videos_limpos['view_count'].sum()
total_likes = df_videos_limpos['like_count'].sum()

print("Pegando das tabelas videos e comentários")
print(f"Total de comentários: {total_comentarios}")
print(f"Total de visualizações: {total_views}")
print(f"Total de curtidas: {total_likes}")


total_comentarios = tabelona['Total comentários'].sum()
total_views = tabelona['Total visualizações'].sum()
total_likes = tabelona['Total curtidas'].sum()

print("")
print("Pegando das tabelona")
print(f"Total de comentários: {total_comentarios}")
print(f"Total de visualizações: {total_views}")
print(f"Total de curtidas: {total_likes}")
print("\n")


prop_curtidas = total_views / total_likes
print(f"Proporção de curtidas em relação às visualizações: {prop_curtidas}")
prop_comentarios = total_views / total_comentarios
print(f"Proporção de comentários em relação às visualizações: {prop_comentarios}")

Pegando das tabelas videos e comentários
Total de comentários: 2238836
Total de visualizações: 813502483
Total de curtidas: 49629553

Pegando das tabelona
Total de comentários: 2238836
Total de visualizações: 813502483
Total de curtidas: 49629553


Proporção de curtidas em relação às visualizações: 16.39149324999965
Proporção de comentários em relação às visualizações: 363.35956854365395


# Tabelona

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from scipy.stats import spearmanr #pegando apenas a funcao spearmanr
import re # Para processar a string de duração ISO 8601

from google.colab import drive
drive.mount('/content/drive')

folder_path = '/content/drive/MyDrive/red_pill_analises/'

Mounted at /content/drive


In [2]:
tabelona = pd.read_csv(folder_path + 'data_base/tabelona_final.csv')

In [ ]:
# Que canal é esse?
df_videos_brutos = pd.read_csv(folder_path + 'data_base/dados_brutos/videos_brutos.csv')
df_channels_brutos = pd.read_csv(folder_path + 'data_base/dados_brutos/canais_brutos.csv')

df_merged = pd.merge(df_videos_brutos[['channel_id', 'video_id']], df_channels_brutos[['channel_id', 'title']], on='channel_id', how='left')

df = df_merged.groupby(['channel_id', 'title']).size().sort_values(ascending=False).reset_index().reset_index()

df['index'] = df['index'] + 1
df

,index,channel_id,title,0
0,1,UCpz_R2L_jolO-RG0V_KvyPQ,Manual AntiOtário,2134
1,2,UCpeW2LVGeNLnSXCp64b3TEQ,Império AntiOtário,1698
2,3,UCdVVI9mJUzGWShsD80poZMg,Fabrício Acunha Mentor,1498
3,4,UCRmNflJuD1TxLbRlDV08_7g,Consciência Masculina,1277
4,5,UCI8GhdRx2zMoAlg-NvwRLog,Top Sedução e Relacionamentos,1147
5,6,UCfx6e_bcnSL0aMDVx0xSlPw,Thiago Schutz (Oficial),1065
6,7,UCZLKzB_7kUljL-A4TojxMmQ,Homem Alpha,992
7,8,UCoigiav3LDJJkt2SiXKxUaQ,Controle AntiOtário,960
8,9,UCUBeVY6Kn7ulBmUGynJqISw,Highlander o último redpill!,808
9,10,UCeL1a4rpEA8UG9IQIewPccg,RedCast [Oficial],709


In [ ]:
tabelona.sort_values(by = 'Inscritos', ascending=False).reset_index(drop=True)[:10]

,id_canal_anonimizado,Inscritos,Vídeos coletados,Meses de atividade,Média vídeos/mês (dp),Duração média (s) (dp),Total visualizações,Média visualizações (dp),Total curtidas,Média likes (dp),Total comentários,Média comentários (dp)
0,canal_19,890000,301,18,16.7 (7.9),873.6 (424.8),5383422,17885.1 (27198.9),666183,2213.2 (2836.3),36739,122.1 (221.0)
1,canal_16,435000,448,18,24.9 (16.7),49.5 (33.2),114572778,255742.8 (827169.1),7141192,15940.2 (46120.5),297920,665.0 (981.4)
2,canal_5,423000,1147,18,63.7 (13.1),52.3 (28.4),78657146,68576.4 (435370.4),5002954,4361.8 (20002.6),143482,125.1 (419.5)
3,canal_1,325000,2134,18,118.6 (48.3),94.7 (107.1),102759571,48153.5 (165719.5),8833125,4139.2 (14375.0),332305,155.7 (332.5)
4,canal_10,320000,709,18,39.4 (23.0),5067.2 (4430.6),33125726,46721.8 (217032.8),2137261,3014.5 (15724.0),132537,186.9 (457.7)
5,canal_15,250000,453,18,25.2 (4.1),681.4 (518.1),19805561,43720.9 (49987.2),2103546,4643.6 (4396.6),210594,464.9 (433.2)
6,canal_2,211000,1698,15,113.2 (42.1),75.6 (42.5),141435162,83295.1 (301671.7),11832309,6968.4 (23493.0),387503,228.2 (526.9)
7,canal_17,188000,410,18,22.8 (5.8),832.5 (410.3),6188898,15094.9 (7050.9),1199838,2926.4 (991.3),210494,513.4 (219.9)
8,canal_14,125000,500,18,27.8 (10.5),262.4 (311.2),14963322,29926.6 (132856.4),923997,1848.0 (6471.8),11986,24.0 (113.3)
9,canal_21,104000,241,17,14.2 (9.2),824.8 (244.2),3971367,16478.7 (16050.0),441387,1831.5 (1432.4),25918,107.5 (115.4)


In [ ]:
# Top 10 valores totais
# Total visualizações, Total curtidas, Total comentários

tabelona.sort_values(by = 'Total visualizações', ascending=False).reset_index(drop=True)[:10]

,id_canal_anonimizado,Inscritos,Vídeos coletados,Meses de atividade,Média vídeos/mês (dp),Duração média (s) (dp),Total visualizações,Média visualizações (dp),Total curtidas,Média likes (dp),Total comentários,Média comentários (dp),media_valor
0,canal_2,211000,1698,15,113.2 (42.1),75.6 (42.5),141435162,83295.1 (301671.7),11832309,6968.4 (23493.0),387503,228.2 (526.9),228.2
1,canal_7,101000,992,17,58.4 (27.5),108.3 (252.6),126248207,127266.3 (732266.8),101651,102.5 (2254.8),59881,60.4 (254.6),60.4
2,canal_16,435000,448,18,24.9 (16.7),49.5 (33.2),114572778,255742.8 (827169.1),7141192,15940.2 (46120.5),297920,665.0 (981.4),665.0
3,canal_1,325000,2134,18,118.6 (48.3),94.7 (107.1),102759571,48153.5 (165719.5),8833125,4139.2 (14375.0),332305,155.7 (332.5),155.7
4,canal_5,423000,1147,18,63.7 (13.1),52.3 (28.4),78657146,68576.4 (435370.4),5002954,4361.8 (20002.6),143482,125.1 (419.5),125.1
5,canal_11,92700,695,5,139.0 (61.4),68.4 (29.4),65901706,94822.6 (539565.5),3112495,4478.4 (13554.0),66729,96.0 (271.0),96.0
6,canal_10,320000,709,18,39.4 (23.0),5067.2 (4430.6),33125726,46721.8 (217032.8),2137261,3014.5 (15724.0),132537,186.9 (457.7),186.9
7,canal_8,46700,960,9,106.7 (56.2),72.8 (38.3),27007213,28132.5 (188179.9),1821735,1897.6 (13885.8),45674,47.6 (199.5),47.6
8,canal_3,77200,1498,18,83.2 (22.9),116.1 (213.1),26280517,17543.7 (104589.6),1098259,733.2 (3632.5),25418,17.0 (87.2),17.0
9,canal_15,250000,453,18,25.2 (4.1),681.4 (518.1),19805561,43720.9 (49987.2),2103546,4643.6 (4396.6),210594,464.9 (433.2),464.9


In [ ]:
tabelona.sort_values(by = 'Total curtidas', ascending=False).reset_index(drop=True)[:10]

,id_canal_anonimizado,Inscritos,Vídeos coletados,Meses de atividade,Média vídeos/mês (dp),Duração média (s) (dp),Total visualizações,Média visualizações (dp),Total curtidas,Média likes (dp),Total comentários,Média comentários (dp),media_valor
0,canal_2,211000,1698,15,113.2 (42.1),75.6 (42.5),141435162,83295.1 (301671.7),11832309,6968.4 (23493.0),387503,228.2 (526.9),228.2
1,canal_1,325000,2134,18,118.6 (48.3),94.7 (107.1),102759571,48153.5 (165719.5),8833125,4139.2 (14375.0),332305,155.7 (332.5),155.7
2,canal_16,435000,448,18,24.9 (16.7),49.5 (33.2),114572778,255742.8 (827169.1),7141192,15940.2 (46120.5),297920,665.0 (981.4),665.0
3,canal_5,423000,1147,18,63.7 (13.1),52.3 (28.4),78657146,68576.4 (435370.4),5002954,4361.8 (20002.6),143482,125.1 (419.5),125.1
4,canal_11,92700,695,5,139.0 (61.4),68.4 (29.4),65901706,94822.6 (539565.5),3112495,4478.4 (13554.0),66729,96.0 (271.0),96.0
5,canal_10,320000,709,18,39.4 (23.0),5067.2 (4430.6),33125726,46721.8 (217032.8),2137261,3014.5 (15724.0),132537,186.9 (457.7),186.9
6,canal_15,250000,453,18,25.2 (4.1),681.4 (518.1),19805561,43720.9 (49987.2),2103546,4643.6 (4396.6),210594,464.9 (433.2),464.9
7,canal_8,46700,960,9,106.7 (56.2),72.8 (38.3),27007213,28132.5 (188179.9),1821735,1897.6 (13885.8),45674,47.6 (199.5),47.6
8,canal_17,188000,410,18,22.8 (5.8),832.5 (410.3),6188898,15094.9 (7050.9),1199838,2926.4 (991.3),210494,513.4 (219.9),513.4
9,canal_13,59000,540,18,30.0 (4.5),315.0 (259.8),13533089,25061.3 (66584.0),1160032,2148.2 (2257.1),127717,236.5 (230.1),236.5


In [4]:
tabelona.sort_values(by = 'Total comentários', ascending=False).reset_index(drop=True)[:10]

,id_canal_anonimizado,Inscritos,Vídeos coletados,Meses de atividade,Média vídeos/mês (dp),Duração média (s) (dp),Total visualizações,Média visualizações (dp),Total curtidas,Média likes (dp),Total comentários,Média comentários (dp)
0,canal_2,211000,1698,15,113.2 (42.1),75.6 (42.5),141435162,83295.1 (301671.7),11832309,6968.4 (23493.0),387503,228.2 (526.9)
1,canal_1,325000,2134,18,118.6 (48.3),94.7 (107.1),102759571,48153.5 (165719.5),8833125,4139.2 (14375.0),332305,155.7 (332.5)
2,canal_16,435000,448,18,24.9 (16.7),49.5 (33.2),114572778,255742.8 (827169.1),7141192,15940.2 (46120.5),297920,665.0 (981.4)
3,canal_15,250000,453,18,25.2 (4.1),681.4 (518.1),19805561,43720.9 (49987.2),2103546,4643.6 (4396.6),210594,464.9 (433.2)
4,canal_17,188000,410,18,22.8 (5.8),832.5 (410.3),6188898,15094.9 (7050.9),1199838,2926.4 (991.3),210494,513.4 (219.9)
5,canal_5,423000,1147,18,63.7 (13.1),52.3 (28.4),78657146,68576.4 (435370.4),5002954,4361.8 (20002.6),143482,125.1 (419.5)
6,canal_10,320000,709,18,39.4 (23.0),5067.2 (4430.6),33125726,46721.8 (217032.8),2137261,3014.5 (15724.0),132537,186.9 (457.7)
7,canal_13,59000,540,18,30.0 (4.5),315.0 (259.8),13533089,25061.3 (66584.0),1160032,2148.2 (2257.1),127717,236.5 (230.1)
8,canal_11,92700,695,5,139.0 (61.4),68.4 (29.4),65901706,94822.6 (539565.5),3112495,4478.4 (13554.0),66729,96.0 (271.0)
9,canal_7,101000,992,17,58.4 (27.5),108.3 (252.6),126248207,127266.3 (732266.8),101651,102.5 (2254.8),59881,60.4 (254.6)


In [ ]:
# Top 10 valores médios
# Média visualizações (dp), Média likes (dp), Média comentários (dp)

tabelona['media_valor'] = tabelona['Média visualizações (dp)'].str.split(' ').str[0].astype(float)

tabelona.sort_values(by='media_valor', ascending=False)[:10]

,id_canal_anonimizado,Inscritos,Vídeos coletados,Meses de atividade,Média vídeos/mês (dp),Duração média (s) (dp),Total visualizações,Média visualizações (dp),Total curtidas,Média likes (dp),Total comentários,Média comentários (dp),media_valor
1,canal_16,435000,448,18,24.9 (16.7),49.5 (33.2),114572778,255742.8 (827169.1),7141192,15940.2 (46120.5),297920,665.0 (981.4),255742.8
10,canal_7,101000,992,17,58.4 (27.5),108.3 (252.6),126248207,127266.3 (732266.8),101651,102.5 (2254.8),59881,60.4 (254.6),127266.3
11,canal_11,92700,695,5,139.0 (61.4),68.4 (29.4),65901706,94822.6 (539565.5),3112495,4478.4 (13554.0),66729,96.0 (271.0),94822.6
6,canal_2,211000,1698,15,113.2 (42.1),75.6 (42.5),141435162,83295.1 (301671.7),11832309,6968.4 (23493.0),387503,228.2 (526.9),83295.1
2,canal_5,423000,1147,18,63.7 (13.1),52.3 (28.4),78657146,68576.4 (435370.4),5002954,4361.8 (20002.6),143482,125.1 (419.5),68576.4
3,canal_1,325000,2134,18,118.6 (48.3),94.7 (107.1),102759571,48153.5 (165719.5),8833125,4139.2 (14375.0),332305,155.7 (332.5),48153.5
4,canal_10,320000,709,18,39.4 (23.0),5067.2 (4430.6),33125726,46721.8 (217032.8),2137261,3014.5 (15724.0),132537,186.9 (457.7),46721.8
13,canal_23,76400,199,10,19.9 (16.7),46.6 (37.3),8992426,45188.1 (123076.2),554086,2784.4 (7277.3),19275,96.9 (227.5),45188.1
5,canal_15,250000,453,18,25.2 (4.1),681.4 (518.1),19805561,43720.9 (49987.2),2103546,4643.6 (4396.6),210594,464.9 (433.2),43720.9
8,canal_14,125000,500,18,27.8 (10.5),262.4 (311.2),14963322,29926.6 (132856.4),923997,1848.0 (6471.8),11986,24.0 (113.3),29926.6


In [ ]:
tabelona['media_valor'] = tabelona['Média likes (dp)'].str.split(' ').str[0].astype(float)

tabelona.sort_values(by='media_valor', ascending=False)[:10]

,id_canal_anonimizado,Inscritos,Vídeos coletados,Meses de atividade,Média vídeos/mês (dp),Duração média (s) (dp),Total visualizações,Média visualizações (dp),Total curtidas,Média likes (dp),Total comentários,Média comentários (dp),media_valor
1,canal_16,435000,448,18,24.9 (16.7),49.5 (33.2),114572778,255742.8 (827169.1),7141192,15940.2 (46120.5),297920,665.0 (981.4),15940.2
6,canal_2,211000,1698,15,113.2 (42.1),75.6 (42.5),141435162,83295.1 (301671.7),11832309,6968.4 (23493.0),387503,228.2 (526.9),6968.4
5,canal_15,250000,453,18,25.2 (4.1),681.4 (518.1),19805561,43720.9 (49987.2),2103546,4643.6 (4396.6),210594,464.9 (433.2),4643.6
11,canal_11,92700,695,5,139.0 (61.4),68.4 (29.4),65901706,94822.6 (539565.5),3112495,4478.4 (13554.0),66729,96.0 (271.0),4478.4
2,canal_5,423000,1147,18,63.7 (13.1),52.3 (28.4),78657146,68576.4 (435370.4),5002954,4361.8 (20002.6),143482,125.1 (419.5),4361.8
3,canal_1,325000,2134,18,118.6 (48.3),94.7 (107.1),102759571,48153.5 (165719.5),8833125,4139.2 (14375.0),332305,155.7 (332.5),4139.2
4,canal_10,320000,709,18,39.4 (23.0),5067.2 (4430.6),33125726,46721.8 (217032.8),2137261,3014.5 (15724.0),132537,186.9 (457.7),3014.5
7,canal_17,188000,410,18,22.8 (5.8),832.5 (410.3),6188898,15094.9 (7050.9),1199838,2926.4 (991.3),210494,513.4 (219.9),2926.4
13,canal_23,76400,199,10,19.9 (16.7),46.6 (37.3),8992426,45188.1 (123076.2),554086,2784.4 (7277.3),19275,96.9 (227.5),2784.4
0,canal_19,890000,301,18,16.7 (7.9),873.6 (424.8),5383422,17885.1 (27198.9),666183,2213.2 (2836.3),36739,122.1 (221.0),2213.2


In [ ]:
tabelona['media_valor'] = tabelona['Média comentários (dp)'].str.split(' ').str[0].astype(float)

tabelona.sort_values(by='media_valor', ascending=False)[:10]

,id_canal_anonimizado,Inscritos,Vídeos coletados,Meses de atividade,Média vídeos/mês (dp),Duração média (s) (dp),Total visualizações,Média visualizações (dp),Total curtidas,Média likes (dp),Total comentários,Média comentários (dp),media_valor
1,canal_16,435000,448,18,24.9 (16.7),49.5 (33.2),114572778,255742.8 (827169.1),7141192,15940.2 (46120.5),297920,665.0 (981.4),665.0
7,canal_17,188000,410,18,22.8 (5.8),832.5 (410.3),6188898,15094.9 (7050.9),1199838,2926.4 (991.3),210494,513.4 (219.9),513.4
5,canal_15,250000,453,18,25.2 (4.1),681.4 (518.1),19805561,43720.9 (49987.2),2103546,4643.6 (4396.6),210594,464.9 (433.2),464.9
14,canal_13,59000,540,18,30.0 (4.5),315.0 (259.8),13533089,25061.3 (66584.0),1160032,2148.2 (2257.1),127717,236.5 (230.1),236.5
6,canal_2,211000,1698,15,113.2 (42.1),75.6 (42.5),141435162,83295.1 (301671.7),11832309,6968.4 (23493.0),387503,228.2 (526.9),228.2
4,canal_10,320000,709,18,39.4 (23.0),5067.2 (4430.6),33125726,46721.8 (217032.8),2137261,3014.5 (15724.0),132537,186.9 (457.7),186.9
3,canal_1,325000,2134,18,118.6 (48.3),94.7 (107.1),102759571,48153.5 (165719.5),8833125,4139.2 (14375.0),332305,155.7 (332.5),155.7
19,canal_25,12700,160,18,8.9 (2.6),839.3 (415.2),973025,6081.4 (6425.2),96842,605.3 (492.9),22153,138.5 (129.5),138.5
20,canal_24,10900,185,11,16.8 (6.8),1481.9 (3029.4),1901023,10275.8 (32393.0),118107,638.4 (1116.0),23621,127.7 (163.1),127.7
2,canal_5,423000,1147,18,63.7 (13.1),52.3 (28.4),78657146,68576.4 (435370.4),5002954,4361.8 (20002.6),143482,125.1 (419.5),125.1
